In [ ]:
# def save_scene_images():
#     for idx, row in tqdm(df.iterrows(), total=len(df)):
#         img_path = os.path.join(findingemo_dir, row['image_path'])
#         boxes = df_boxes[df_boxes['index'] == idx][['x1', 'y1', 'x2', 'y2']].values.tolist()
#         blurred_image = blur_faces(img_path, boxes)
#         blurred_image = cv2.resize(blurred_image, (224, 224))
#         save_path = os.path.join(findingemo_dir, "scenes_5", f"scene_{idx}.jpg")
#         cv2.imwrite(save_path, blurred_image, [cv2.IMWRITE_JPEG_QUALITY, 90])

# save_scene_images()

In [ ]:
# faces_dir = os.path.join(findingemo_dir, "faces")
# os.makedirs(faces_dir, exist_ok=True)

# csv_path = os.path.join(findingemo_dir, "face_boxes.csv")

# # Create and open CSV file for writing
# with open(csv_path, "w", newline="") as f:
#     writer = csv.writer(f)
#     # header row
#     writer.writerow(["index", "image_path", "face_rank", "x1", "y1", "x2", "y2", "score", "crop_path"])

#     # ========================================
#     # Iterate over the dataframe and process each image
#     # ========================================
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Detecting faces"):
#         img_path = os.path.join(findingemo_dir, row["image_path"])

#         try:
#             detections = RetinaFace.detect_faces(img_path)
#         except Exception as e:
#             print(f"[Warning] Could not process {img_path}: {e}")
#             continue

#         # Skip images without faces
#         if not detections or isinstance(detections, str):
#             continue

#         # Collect all detected faces
#         faces = []
#         for det in detections.values():
#             x1, y1, x2, y2 = det["facial_area"]
#             area = (x2 - x1) * (y2 - y1)
#             faces.append({
#                 "coords": (x1, y1, x2, y2),
#                 "score": det["score"],
#                 "area": area
#             })

#         # Sort faces by area (largest first) and keep top 3
#         faces = sorted(faces, key=lambda x: x["area"], reverse=True)[:3]

#         # Crop and save top faces
#         img = Image.open(img_path).convert("RGB")
#         for i, face in enumerate(faces):
#             x1, y1, x2, y2 = face["coords"]
#             crop = img.crop((x1, y1, x2, y2)).resize((224, 224))
#             crop_name = f"img_{idx}_face_{i}.jpg"
#             crop_path = os.path.join(faces_dir, crop_name)
#             crop.save(crop_path, "JPEG", quality=90)

#             # Write bounding box info to CSV
#             writer.writerow([
#                 idx,                      # image index
#                 row["image_path"],        # relative image path
#                 i,                        # face rank (0=largest)
#                 x1, y1, x2, y2,           # bounding box coordinates
#                 face["score"],            # detection confidence
#                 crop_path                 # saved crop file
#             ])

# print(f"Finished! Cropped faces saved in: {faces_dir}")
# print(f"Bounding box CSV saved at: {csv_path}")


In [ ]:
# def save_pretrained_logits_separately(indices, output_dir):
#     """
#     Save pre-computed logits from pretrained models to separate files.
    
#     Args:
#         indices: list of sample indices to process
#         output_dir: base directory to save the logits
#     """
#     # Create output directories
#     scene_dir = os.path.join(output_dir, "scenes")
#     faces_dir = os.path.join(output_dir, "faces")
#     os.makedirs(scene_dir, exist_ok=True)
#     os.makedirs(faces_dir, exist_ok=True)
    
#     # Set models to eval mode
#     face_model_base.eval()
#     scene_model_base.eval()
    
#     for img_idx in tqdm(indices, desc="Saving pretrained logits"):
#         # ----- PROCESS SCENE -----
#         scene_img = scenes_dataset[(img_idx,)].to(DEVICE).unsqueeze(0)  # (1, C, H, W)
        
#         with torch.no_grad():
#             scene_logits = scene_model_base(scene_img).squeeze(0)  # (365,)
        
#         # Save scene logits
#         scene_file = os.path.join(scene_dir, f"scene_{img_idx}.pt")
#         torch.save(scene_logits.cpu(), scene_file)
        
#         # ----- PROCESS FACES -----
#         face_rows = df_boxes[df_boxes['index'] == img_idx].sort_values('face_rank')
        
#         for _, row in face_rows.iterrows():
#             face_rank = int(row['face_rank'])
#             face_tensor = faces_dataset[(img_idx, face_rank)]
            
#             with torch.no_grad():
#                 face_tensor = face_tensor.to(DEVICE).unsqueeze(0)  # (1, C, H, W)
#                 outputs = face_model_base(face_tensor)  # Returns ImageClassifierOutput
#                 face_logits = outputs.logits.squeeze(0)  # (7,)
            
#             # Save face logits
#             face_file = os.path.join(faces_dir, f"face_{img_idx}_{face_rank}.pt")
#             torch.save(face_logits.cpu(), face_file)
    
#     print(f"\nLogits saved to:")
#     print(f"  Scenes: {scene_dir}")
#     print(f"  Faces: {faces_dir}")

# # Usage:
# # save_pretrained_logits_separately(indices, output_dir=os.path.join(data_dir, "pretrained_logits"))

In [ ]:
# def save_pretrained_logits_separately(indices, output_dir):
#     """
#     Save pre-computed logits from pretrained models to separate files.
    
#     Args:
#         indices: list of sample indices to process
#         output_dir: base directory to save the logits
#     """
#     # Create output directories
#     scene_dir = os.path.join(output_dir, "scenes")
#     os.makedirs(scene_dir, exist_ok=True)
    
#     # Set models to eval mode
#     scene_model_base.eval()
    
#     for img_idx in tqdm(indices, desc="Saving pretrained logits"):
#         # ----- PROCESS SCENE -----
#         scene_img = scenes_dataset[img_idx].to(DEVICE).unsqueeze(0)  # (1, C, H, W)
        
#         with torch.no_grad():
#             scene_logits = scene_model_base(scene_img)
#             scene_logits = scene_logits.view(scene_logits.size(0), -1)
        
#         # Save scene logits
#         scene_file = os.path.join(scene_dir, f"scene_{img_idx}.pt")
#         torch.save(scene_logits.cpu(), scene_file)
        
#     print(f"\nEmbeddings saved to:")
#     print(f"  Scenes: {scene_dir}")

# # Usage:
# save_pretrained_logits_separately(indices, output_dir=os.path.join(data_dir, "pretrained_backbone"))

In [ ]:
# # Show the image but with faces blurred out
# def blur_faces(image_path, boxes):
#     image = cv2.imread(image_path)
#     for box in boxes:
#         x1, y1, x2, y2 = box
#         face = image[y1:y2, x1:x2]
#         blurred_face = cv2.GaussianBlur(face, (99, 99), 30)
#         image[y1:y2, x1:x2] = blurred_face
#     return image

# img_path = os.path.join(findingemo_dir, df.iloc[9]['image_path'])
# boxes = df_boxes[df_boxes['index'] == 9][['x1', 'y1', 'x2', 'y2']].values.tolist()
# blurred_image = blur_faces(img_path, boxes)

# plt.imshow(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

In [ ]:
# def show_image(dataframe, index):
#     img_path = os.path.join(findingemo_dir, dataframe.loc[index, 'image_path'])
#     print("Image path:", img_path)
#     image = cv2.imread(img_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()

In [ ]:
face_val_model_dpl = CORNWrapper(face_val_model, num_classes=len(VALENCE_BINS))


In [ ]:
def get_va_model_predictions(model, idx, dataset):
    model.eval()
    with torch.no_grad():
        x, y = dataset[idx]
        x = x.to(DEVICE).float().unsqueeze(0)  # Add batch dimension
        y = y

        probs = model(x)
        print(probs)
        print("")
        print(df[df.index == test_indices[idx]][['valence_bin', 'arousal_bin']])
        show_image(df, test_indices[idx])

get_va_model_predictions(face_val_model_dpl, random.randint(0, len(face_val_test_dataset)-1), face_val_test_dataset)
# get_va_model_predictions(face_val_model_dpl, 0, face_val_test_dataset)

In [ ]:
class CORNWrapper(nn.Module):
    def __init__(self, base_model, num_classes):
        super().__init__()
        self.base_model = base_model
        self.num_classes = num_classes

    def forward(self, x):
        logits = self.base_model(x)                # (B, K-1)
        p_gt = torch.sigmoid(logits)               # threshold probs

        B = logits.size(0)
        probs = torch.zeros(B, self.num_classes, device=logits.device)

        # P(y = 0)
        probs[:, 0] = 1 - p_gt[:, 0]

        # P(y = k)
        for k in range(1, self.num_classes - 1):
            probs[:, k] = p_gt[:, k-1] - p_gt[:, k]

        # P(y = K)
        probs[:, -1] = p_gt[:, -1]

        # Fix negatives
        probs = torch.clamp(probs, min=0)

        # Normalize
        probs = probs / probs.sum(dim=1, keepdim=True)

        return probs


In [ ]:
def show_image(dataframe, index):
    img_path = os.path.join(findingemo_dir, dataframe.loc[index, 'image_path'])
    print("Image path:", img_path)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.axis('off')
    plt.show()